In [8]:
MODEL_ID='COHA_byhalfcentury_nonf_smpl'
MODEL_FN='static/data/db/models/COHA_byhalfcentury_nonf/chained_combined/1800-1999.min=500.run=01.txt'

In [9]:
SAVE_N_TOP = 100

# Build DB

In [24]:
import os,gensim,networkx as nx
from tqdm import tqdm
from scipy.spatial.distance import squareform,pdist
import pandas as pd

In [11]:
MODEL_DIR = os.path.join('db',MODEL_ID)
if not os.path.exists(MODEL_DIR): os.makedirs(MODEL_DIR)

## Load gensim

In [12]:
import gensim
M = gensim.models.KeyedVectors.load_word2vec_format(MODEL_FN)

In [13]:
WORDS = list(M.vocab)
VECS = M.vectors

### Add fields/etc?

## Build dist db

In [14]:
def get_dists(vecs,words,n_top=SAVE_N_TOP):
    print('>> getting distance matrix...')
    distsX = squareform(pdist(vecs,metric='cosine'))
    print('>> done getting distance matrix...')
    return pd.DataFrame(distsX,index=words,columns=words)

In [15]:
dfdist = get_dists(VECS,WORDS)

>> getting distance matrix...
>> done getting distance matrix...


In [27]:
def build_dist_db(n_top=SAVE_N_TOP):    
    print('>> graphing...')
    ofn='db/'+MODEL_ID+'/data.simnet.csv'
    with open(ofn,'w') as of:
        of.write('source,target,weight,sim_rank\n')
        
        G=nx.Graph()
        
        for word in tqdm(WORDS):
            row = dfdist.loc[word].sort_values().iloc[1:n_top+1]
            sim_rank=0
            for word2,result in zip(row.index,row):
                sim_rank+=1
                G.add_edge(word,word2,weight=result,sim_rank=sim_rank)
        
        
        for a,b,d in tqdm(G.edges(data=True)):
            row=[a,b,d['weight'],d['sim_rank']]
            rowstr=','.join([str(x) for x in row])
            of.write(rowstr+'\n')
        
        print('>> saved:')

In [28]:
build_dist_db()


  0%|          | 21/31228 [00:00<02:33, 203.64it/s]

>> graphing...



100%|██████████| 2217449/2217449 [00:12<00:00, 179387.38it/s]

>> saved:


## Build vec db

In [29]:
def build_vec_db():
    from pymongo import MongoClient
    client = MongoClient()
    vecdb = client['vecdb'][MODEL_ID]
    
    for word in tqdm(WORDS):
        wvecs = [float(x) for x in M[word]]
        doc = {'word':word, 'vecs':wvecs}
        vecdb.insert(doc)

In [30]:
build_vec_db()


  0%|          | 0/31228 [00:00<?, ?it/s]/Users/ryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':

100%|██████████| 31228/31228 [00:21<00:00, 1462.20it/s]

In [33]:
# !wc -l db/COHA_byhalfcentury_nonf_smpl/data.simnet.csv
# !ls -ltrh db/COHA_byhalfcentury_nonf_smpl/data.simnet.csv